In [1]:
from datasets import load_dataset
import polars as pl
import os
import cynde.functional as cf

In [2]:
# Get the directory above the current directory
base_path = os.path.dirname(os.getcwd())

# Specify the cache directory as 'data' subdirectory within the current directory
cache_dir = os.path.join(base_path, "cache")
print(cache_dir)
print(base_path)

c:\Users\Tommaso\Documents\Dev\Cynde\cache
c:\Users\Tommaso\Documents\Dev\Cynde


In [3]:
dataset_name = "OpenHermes-2.5"
dataset_path = os.path.join(cache_dir, dataset_name)


In [4]:
df = pl.read_parquet(dataset_path+"_filtered.parquet")
df.head(1)


conversations,source
list[struct[3]],str
"[{""human"",""Given the following list of words, categorize them into 5 categories by similarity: ['Heart', 'Liver', 'Kidney', 'Brain', 'Stomach', 'Skin', 'Lung', 'Muscle', 'Intestine', 'Bone', 'Pancreas', 'Blood', 'Eye', 'Ear', 'Nose', 'Mouth', 'Esophagus', 'Throat', 'Spinal Cord', 'Bladder']"",null}, {""gpt"",""Here are the given words categorized into 5 groups based on their similarity: 1. Vital Organs: - Heart - Liver - Kidney - Lung - Pancreas 2. Digestive System Components: - Stomach - Intestine - Esophagus - Throat - Mouth 3. Sensory Organs: - Eye - Ear - Nose 4. Support and Movement: - Bone - Muscle - Spinal Cord - Skin 5. Body Fluids and Waste Management: - Blood - Bladder"",null}, … {""gpt"",""1) Vital Organs: - Diseases or disorders affecting one vital organ can impact the functioning of others. For example, heart failure can lead to fluid buildup in the lungs, affecting their function. Liver disease can cause toxins to accumulate in the bloodstream, potentially affecting brain function. Kidney disease can lead to electrolyte imbalances, which can impact heart function. - Organ transplantation involves replacing a damaged organ with a healthy one from a donor. Examples include heart, liver, kidney, lung, and pancreas transplants. Challenges include finding compatible donors, managing immune system rejection, and addressing surgical complications. 2) Digestive System Components: - Common digestive disorders include gastroesophageal reflux disease (GERD), irritable bowel syndrome (IBS), inflammatory bowel disease (IBD), and peptic ulcers. Causes may include genetic factors, lifestyle choices, stress, infections, or immune system dysfunction. - The gut microbiome comprises trillions of microorganisms that aid in digestion, nutrient absorption, and immune system function. It can also influence mental health and contribute to the prevention or development of various diseases. 3) Sensory Organs: - Optical illusions work by exploiting the brain's visual processing system, causing it to misinterpret sensory information. They reveal that our perception of reality can be influenced by factors such as context, contrast, and patterns. - Animals' sensory organs can differ significantly from humans', offering unique capabilities. For example, dogs have a heightened sense of smell, eagles have exceptional long-distance vision, and bats use echolocation for navigation and foraging. 4) Support and Movement: - Exercise and physical activity promote bone and muscle health by stimulating bone growth, increasing bone density, and strengthening muscles. This helps prevent age-related bone loss and reduces the risk of injury. - Spinal cord injuries can have varying effects depending on the location and severity of the injury. Injuries can result in temporary or permanent loss of sensation, mobility, and other bodily functions. Paraplegia (paralysis of the legs and lower body) and quadriplegia (paralysis of all four limbs) are examples of potential outcomes of spinal cord injuries. 5) Body Fluids and Waste Management: - Blood types affect transfusions and organ transplants because of the presence of specific antigens on red blood cells. Incompatible blood types can cause an immune response, resulting in transfusion reactions or organ rejection. - Common bladder disorders include urinary incontinence, urinary tract infections (UTIs), interstitial cystitis, and bladder cancer. Treatments and management strategies can vary, including medication, lifestyle changes, physical therapy, or surgical interventions."",null}]","""caseus_custom"""


In [5]:
df["source"].value_counts()

source,count
str,u32
"""LMSys Chatbot …",3136
"""caseus_custom""",2688


In [6]:
df = df.with_columns(pl.col("conversations").list.eval(pl.element().struct.json_encode()).list.join("\n"),(pl.col("source")=="caseus_custom").alias("target"))

In [7]:
#check if the embedded df already exist at dataset_path+"_embedded.parquet"
if not os.path.exists(dataset_path+"_embedded_small_large.parquet"):
    embedded_df = cf.embed_columns(df, ["conversations"],models=["text-embedding-3-small","text-embedding-3-large"])
    embedded_df.write_parquet(dataset_path+"_embedded_small_large.parquet")
else:
    embedded_df = pl.read_parquet(dataset_path+"_embedded_small_large.parquet")

In [8]:
df["target"].value_counts()

target,count
bool,u32
false,3136
true,2688


In [9]:
cf.vanilla_kfold(embedded_df,group=None,k=5)

cv_index,fold_0,fold_1,fold_2,fold_3,fold_4
u32,str,str,str,str,str
2011,"""test""","""train""","""train""","""train""","""train"""
2951,"""test""","""train""","""train""","""train""","""train"""
2920,"""test""","""train""","""train""","""train""","""train"""
1551,"""test""","""train""","""train""","""train""","""train"""
2864,"""test""","""train""","""train""","""train""","""train"""
5201,"""test""","""train""","""train""","""train""","""train"""
3533,"""test""","""train""","""train""","""train""","""train"""
5577,"""test""","""train""","""train""","""train""","""train"""
2482,"""test""","""train""","""train""","""train""","""train"""


df:pl.DataFrame, cv_type: Tuple[str,str], group_outer:List[str],k_outer:int,group_inner:List[str],k_inner:int,r_outer:int =1, r_inner:int =1):

In [10]:
models_dict = {"RandomForest": [{"n_estimators": 10, "max_depth": 5},{"n_estimators": 50, "max_depth": 10}]}
# models_dict = {"RandomForest": [{"n_estimators": 10, "max_depth": 5}]}
inputs =[{"numerical":["conversations_text-embedding-3-small_embeddings"]},
         {"numerical":["conversations_text-embedding-3-large_embeddings"]},
         {"numerical":["conversations_text-embedding-3-small_embeddings","conversations_text-embedding-3-large_embeddings"]}]
# inputs = [inputs[0]]


In [11]:
# results,pred=cf.train_nested_cv(df = embedded_df,
#                      cv_type=("stratified","stratified"),
#                      inputs=inputs,
#                      models=models_dict,
#                      group_outer=["target"],
#                      k_outer = 2,
#                      group_inner=["target"],
#                      k_inner = 2,
#                      r_outer=1,
#                      r_inner=1,
#                      save_name="test",
#                      base_path=base_path,)


In [12]:
# summary = cf.results_summary(results,by_test_fold=True)
# summary

In [13]:
# for res in summary.sort(by="mcc_val",descending=True).rows(named=True):
#     print(res["input_features_name"],res["classifier_hp"],res["mcc_val"])

In [14]:
# results,pred=cf.train_nested_cv_simple(df = embedded_df,
#                      cv_type=("stratified","stratified"),
#                      inputs=inputs,
#                      models=models_dict,
#                      group_outer=["target"],
#                      k_outer = 2,
#                      group_inner=["target"],
#                      k_inner = 2,
#                      r_outer=1,
#                      r_inner=1,
#                      save_name="test",
#                      base_path=base_path,)

In [15]:
# summary = cf.results_summary(results,by_test_fold=True)
# summary

In [23]:
results,pred=cf.train_nested_cv_simple(df = embedded_df,
                     cv_type=("stratified","stratified"),
                     inputs=inputs,
                     models=models_dict,
                     group_outer=["target"],
                     k_outer = 2,
                     group_inner=["target"],
                     k_inner = 2,
                     r_outer=10,
                     r_inner=1,
                     save_name="test",
                     base_path=base_path,
                     skip_class=True)# 

cv_type: stratified
cv_type: stratified
cv_creation_time: 0.10102152824401855
fit_time: 138.0366189479828
average time per fold: 3.4509154736995695
aggregation_time: 0.0004992485046386719
Saving results to c:\Users\Tommaso\Documents\Dev\Cynde\data_processed\results_2024-02-26_15-46-42_test.parquet
save_time: 2.110860824584961
total_time: 140.2490005493164
Total average time per fold: 3.50622501373291


In [21]:
results,pred=cf.train_nested_cv_simple(df = embedded_df,
                     cv_type=("stratified","stratified"),
                     inputs=inputs,
                     models=models_dict,
                     group_outer=["target"],
                     k_outer = 2,
                     group_inner=["target"],
                     k_inner = 2,
                     r_outer=10,
                     r_inner=1,
                     save_name="test",
                     base_path=base_path,
                     skip_class=False)# 

cv_type: stratified
cv_type: stratified
cv_creation_time: 0.1010141372680664
fit_time: 455.80155086517334
average time per fold: 11.395038771629334
aggregation_time: 0.130018949508667
Saving results to c:\Users\Tommaso\Documents\Dev\Cynde\data_processed\results_2024-02-26_15-37-37_test.parquet
save_time: 4.233220815658569
total_time: 460.26580476760864
Total average time per fold: 11.506645119190216


In [25]:
results,pred=cf.train_nested_cv_from_np(df = embedded_df,
                     cv_type=("stratified","stratified"),
                     inputs=inputs,
                     models=models_dict,
                     group_outer=["target"],
                     k_outer = 2,
                     group_inner=["target"],
                     k_inner = 2,
                     r_outer=10,
                     r_inner=1,
                     save_name="test",
                     base_path=base_path,
                     skip_class=True)

cv_type: stratified
cv_type: stratified
results schema:  OrderedDict([('classifier', String), ('classifier_hp', String), ('fold_name', String), ('pred_name', String), ('input_features_name', String), ('accuracy_train', Float64), ('accuracy_val', Float64), ('accuracy_test', Float64), ('mcc_train', Float64), ('mcc_val', Float64), ('mcc_test', Float64), ('train_index', List(UInt32)), ('val_index', List(UInt32)), ('test_index', List(UInt32)), ('train_time', String), ('pred_time', String), ('eval_time', String), ('total_cls_time', String), ('k_outer', Int64), ('k_inner', Int64), ('r_outer', Int64), ('r_inner', Int64)])
results shape: (0, 22)
Feature array shape for conversations_text-embedding-3-small_embeddings: (5824, 1536)
Feature array shape for conversations_text-embedding-3-large_embeddings: (5824, 3072)
Feature array shape for conversations_text-embedding-3-small_embeddings_conversations_text-embedding-3-large_embeddings: (5824, 4608)
Preprocessing completed in 3.3631350994110107 sec

In [22]:
results,pred=cf.train_nested_cv_from_np(df = embedded_df,
                     cv_type=("stratified","stratified"),
                     inputs=inputs,
                     models=models_dict,
                     group_outer=["target"],
                     k_outer = 2,
                     group_inner=["target"],
                     k_inner = 2,
                     r_outer=10,
                     r_inner=1,
                     save_name="test",
                     base_path=base_path,
                     skip_class=False)

cv_type: stratified
cv_type: stratified
results schema:  OrderedDict([('classifier', String), ('classifier_hp', String), ('fold_name', String), ('pred_name', String), ('input_features_name', String), ('accuracy_train', Float64), ('accuracy_val', Float64), ('accuracy_test', Float64), ('mcc_train', Float64), ('mcc_val', Float64), ('mcc_test', Float64), ('train_index', List(UInt32)), ('val_index', List(UInt32)), ('test_index', List(UInt32)), ('train_time', String), ('pred_time', String), ('eval_time', String), ('total_cls_time', String), ('k_outer', Int64), ('k_inner', Int64), ('r_outer', Int64), ('r_inner', Int64)])
results shape: (0, 22)
Feature array shape for conversations_text-embedding-3-small_embeddings: (5824, 1536)
Feature array shape for conversations_text-embedding-3-large_embeddings: (5824, 3072)
Feature array shape for conversations_text-embedding-3-small_embeddings_conversations_text-embedding-3-large_embeddings: (5824, 4608)
Preprocessing completed in 3.4060003757476807 sec

In [18]:
summary = cf.results_summary(results,by_test_fold=True)
summary

classifier,classifier_hp,input_features_name,r_outer,r_inner,mcc_train,mcc_val,mcc_test,accuracy_train,accuracy_val,accuracy_test,n
str,str,str,i64,i64,f64,f64,f64,f64,f64,f64,u32
"""RandomForest""","""n_estimators_5…","""conversations_…",0,0,0.999655,0.876838,0.878368,0.999828,0.938702,0.939475,4
"""RandomForest""","""n_estimators_5…","""conversations_…",0,0,0.999655,0.875123,0.868604,0.999828,0.937843,0.934581,4
"""RandomForest""","""n_estimators_5…","""conversations_…",0,0,0.999655,0.829189,0.836273,0.999828,0.915007,0.918441,4
"""RandomForest""","""n_estimators_1…","""conversations_…",0,0,0.930615,0.77381,0.774594,0.965488,0.887534,0.887964,4
"""RandomForest""","""n_estimators_1…","""conversations_…",0,0,0.928554,0.768298,0.765439,0.964457,0.884787,0.883328,4
"""RandomForest""","""n_estimators_1…","""conversations_…",0,0,0.893987,0.727273,0.720735,0.947287,0.864354,0.861178,4
